
# Для распознавания используется предварительно обученная сверточная нейронная сеть VGG16.

Перед использованием необходимо скачать и подготовить данные для обучения, проверки и тестирования. Можно использовать пример в ноутбуке data_preparation

In [ ]:
# Load the Drive helper and mount
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import time
start_time = time.time()

from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.python.keras.applications import vgg16
from tensorflow.python.keras.optimizers import Adam
import matplotlib.pyplot as plt


In [ ]:
# Каталог с данными для обучения
train_dir = '/content/drive/My Drive/Colab Notebooks/weld_set/ready_ds_254/train_set'
# Каталог с данными для проверки
val_dir = '/content/drive/My Drive/Colab Notebooks/weld_set/ready_ds_254/validation_set'
# Каталог с данными для тестирования
test_dir = '/content/drive/My Drive/Colab Notebooks/weld_set/ready_ds_254/test_set'
# Размеры изображения
img_width, img_height = 128, 128
# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 3)  # 3- каналы для цвета
# Количество классов изображений
nb_classes =5
# Количество эпох
epochs = 70
# Размер мини-выборки
batch_size = 2
# Количество изображений для обучения
nb_train_samples = 890
# Количество изображений для проверки
nb_validation_samples = 190
# Количество изображений для тестирования
nb_test_samples = 190

Загружаем предварительно обученную нейронную сеть

In [ ]:
vgg16_net = vgg16.VGG16(weights='imagenet', # загружаем веса
                  include_top=False, # классификатор не вкл
                  input_shape=(128, 128, 3))

58892288/58889256 [==============================] - 3s 0us/step


"Замораживаем" веса предварительно обученной нейронной сети VGG16

In [ ]:
#vgg19_net.trainable = False # запрещаем обучаться
vgg16_net.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0     

In [ ]:
for layer in vgg16_net.layers[:5]:
    layer.trainable = False
vgg16_net.summary()    

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0     

## Создаем составную нейронную сеть на основе VGG16

In [ ]:
model = Sequential()
# Добавляем в модель сеть VGG19 вместо слоя
model.add(vgg16_net) # загружаем сверточную часть сети

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(1024))
model.add(Activation('relu'))
#model.add(Dropout(0.5))

model.add(Dense(5))
model.add(Activation('softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              8389632   
_________________________________________________________________
activation (Activation)      (None, 1024)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
activation_1 (Activation)    (None, 1024)              0

Компилируем составную нейронную сеть

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(0.001), 
              metrics=["accuracy"])

Стохастический оптимизатор градиентного спуска.
lr: float> = 0. Скорость обучения.
momentum: float> = 0. Параметр, который ускоряет SGD в соответствующем направлении и гасит колебания.

In [ ]:
import tensorflow as tf
model.compile(loss='categorical_crossentropy',
              optimizer= tf.keras.optimizers.SGD(lr=0.0001, momentum=0.9), 
              metrics=["accuracy"])

#Создаем генератор изображений
Генератор изображений создается на основе класса ImageDataGenerator. Генератор делит значения всех пикселов изображения на 255.

In [ ]:
datagen = ImageDataGenerator(rescale=1. / 255)

#Генератор данных для обучения на основе изображений из каталога
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

# для проверки 
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

#для тестирования 
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle = False
    )

Found 890 images belonging to 5 classes.
Found 190 images belonging to 5 classes.
Found 190 images belonging to 5 classes.


In [ ]:
# Создаем EarlyStopping Callback
early_stopping_callback = EarlyStopping(monitor='val_accuracy', patience=2) # если val_acc два раза подряд меньше чем 3 значение с конца то стоп patience=2

NameError: ignored

## Обучаем модель с использованием генераторов

train_generator - генератор данных для обучения

validation_data - генератор данных для проверки

In [ ]:
model.fit_generator(
    train_generator, # с учетом расширения данных
    steps_per_epoch=nb_train_samples // batch_size, # генератор может работать бесконечно, поэтому указываем количество шагов на эпоху // batch_size
    epochs=20,
    validation_data=val_generator, # для проверки
    validation_steps=nb_validation_samples // batch_size # кол-во шагов // batch_size
    ) 

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
445/445 [==============================] - 320s 720ms/step - loss: 1.6684 - accuracy: 0.1820 - val_loss: 1.6096 - val_accuracy: 0.2000
Epoch 2/20
445/445 [==============================] - 25s 56ms/step - loss: 1.6141 - accuracy: 0.2326 - val_loss: 1.6965 - val_accuracy: 0.2000
Epoch 3/20
445/445 [==============================] - 25s 57ms/step - loss: 1.5977 - accuracy: 0.2494 - val_loss: 1.5852 - val_accuracy: 0.2316
Epoch 4/20
445/445 [==============================] - 25s 56ms/step - loss: 1.5315 - accuracy: 0.3022 - val_loss: 1.4440 - val_accuracy: 0.3789
Epoch 5/20
445/445 [==============================] - 24s 55ms/step - loss: 1.3866 - accuracy: 0.3685 - val_loss: 1.3188 - val_accuracy: 0.4421
Epoch 6/20
445/445 [==============================] - 25s 55ms/step - loss: 1.1842 - accuracy: 0.4944 - val_loss: 1.0921 - val_accuracy: 0.5368
Epoch 7/20
445/445 [==============================] - 25s 

## Оцениваем качество работы сети с помощью генератора

In [ ]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print("Аккуратность на тестовых данных: %.2f%%" % (scores[1]*100))
print("--- %s minutes ---" % ((time.time() - start_time) / 60))


# стороит массив класс*кол-во изображений [вероятность принадлежности класса]
# array([[3.2083929e-01, 2.3337396e-02, 6.5582341e-01], -> ответ 2класс 65% веростность
pred = model.predict_generator(test_generator,steps = len(test_generator), verbose = 1)

true_classes = test_generator.classes  # array([0, 0, 0,...1,1,1,...2,2,2,...],dtype=int32)
class_labels = list(test_generator.class_indices.keys())  #['CR', 'GW', 'PO']
from sklearn.metrics import classification_report

## array([[3.2083929e-01, 2.3337396e-02, 6.5582341e-01], -> ответ 2класс 65% веростность
#из трех выбирает больший по горизонтали и записывает индекс максимума
import numpy
predicted_classes = numpy.argmax(pred, axis=1) # axis=1-горизонтально

report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report) 

Instructions for updating:
Please use Model.evaluate, which supports generators.
Аккуратность на тестовых данных: 78.95%
--- 16.14692031145096 minutes ---
Instructions for updating:
Please use Model.predict, which supports generators.
95/95 [==============================] - 2s 19ms/step
                    precision    recall  f1-score   support

         good well       0.97      0.95      0.96        38
longitudinal crack       0.86      0.82      0.84        38
          porosity       0.57      0.97      0.72        38
   slag inclusions       0.70      0.37      0.48        38
  transverse crack       1.00      0.84      0.91        38

          accuracy                           0.79       190
         macro avg       0.82      0.79      0.78       190
      weighted avg       0.82      0.79      0.78       190



In [ ]:
print("Сохраняем сеть")
# Сохраняем сеть для последующего использования
# Генерируем описание модели в формате json
model_json = model.to_json()
json_file = open("/content/drive/My Drive/Colab Notebooks/save_п/weld_vgg16_5_classes254_20_1.json", "w")
# Записываем архитектуру сети в файл
json_file.write(model_json)
json_file.close()
# Записываем данные о весах в файл
model.save_weights("/content/drive/My Drive/Colab Notebooks/save_п/weld_vgg16_5_classes254_20_1.h5")
print("Сохранение сети завершено")
#model.save('/content/drive/My Drive/Colab Notebooks/data_augmentation.h5')

Сохраняем сеть
Сохранение сети завершено
